# Libraries

In [3]:
# !pip install imbalanced-learn
# !pip install scikit-image
# !pip install statsmodels
# !pip install tensorflow
# !pip install pydot
# !pip install graphviz

In [4]:
# import zipfile
# with zipfile.ZipFile("training.zip", 'r') as zip_ref:
#     zip_ref.extractall("")

In [5]:
from wettbewerb import load_references, save_predictions
from preprocess import *
from sklearn import preprocessing
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import pydot
import graphviz

# Data import

In [6]:
ecg_leads, ecg_labels, fs, ecg_names = load_references(folder='/shared_data/training')

6000	 Dateien wurden geladen.


# Preprocessing

In [7]:
ecg_leads_edited = ecg_leads

### Noise removal (fan_multiscaled_2018)

In [8]:
filtered = []
for j, data in enumerate(ecg_leads_edited):
    data_ftt, freq = ecg_furier(ecg_leads_edited[j], fs)
    lowpassed = ecg_denoise_spectrum(data_ftt, freq, 0, 60)
    filtered.append(ecg_invfurier(lowpassed))

ecg_leads_edited = filtered

### Z-normalization

In [9]:
filtered = []
for j, data in enumerate(ecg_leads_edited):
    filtered.append(ecg_norm(ecg_leads_edited[j]))

ecg_leads_edited = filtered

### Length normalization (hsieh_detection_2020)

In [10]:
def ecg_split(data, limit, label):
    splitted = []
    ratio = len(data)/limit

    # If data is longer than limit, split with at least 50% overlap
    if ratio > 1:
        for i in range(0, int(np.ceil(2*ratio))):
            if i == int(np.ceil(2*ratio))-1:
                string = data[len(data)-limit:len(data)]
            else:
                string = data[int(np.floor(i*limit/2)):int(np.floor(i*limit/2+limit))]
            splitted.append(string)
        splitted = [x for x in splitted if x.shape[0] >= limit] # remove string which are shorter
        labels_multiplied = [label] * len(splitted)

    # If data is shorter than limit, add from the beginning
    elif ratio < 1:
        if ratio <= 0.5:
            data = np.tile(data, int(np.floor(1/ratio)))
        diff = limit - len(data)
        appended_data = data[0:diff]
        data = np.append(data, appended_data)
        splitted.append(data)
        labels_multiplied = [label] * 1

    # If it is the exact length, then don't alter it
    elif ratio == 1:
        splitted.append(data)
        labels_multiplied = [label] * 1
    return splitted, labels_multiplied

In [11]:
normalized = []
normalized_label = []
time_set = 20
duration = time_set*fs
for j, data in enumerate(ecg_leads_edited):
    ecg_split_data, lab_mul = ecg_split(data, duration, ecg_labels[j])
    normalized.extend(ecg_split_data)
    normalized_label.extend(lab_mul)

ecg_leads_edited = normalized
ecg_labels_edited = normalized_label

### Split training, validation, test data

In [70]:
# Validation set
X_train, X_test, y_train, y_test = train_test_split(ecg_leads_edited, ecg_labels_edited, test_size=0.2, shuffle = True, random_state = 8)

In [71]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

### Random oversampling for imbalanced training data

In [72]:
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Converts sublists to arrays
xfiltered = []
yfiltered = []
for j, data in enumerate(X_train):
    xfiltered.append(np.asarray(X_resampled[j]))
    yfiltered.append(np.asarray(y_resampled[j]))
X_train = xfiltered
y_train = yfiltered

### Label encoding (0: A, 1: N, 2: O, 3: ~)

In [60]:
# get indices for each label
# idx_N = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'N']
# idx_A = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'A']
# idx_tilde = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == '~']
# idx_O = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'O']

In [77]:
def labelencode(y_data):
    le = preprocessing.LabelEncoder()
    le.fit(y_data)
    encoded = le.transform(y_data)
    encoded = keras.utils.to_categorical(encoded)
    return encoded

def tolist(array):
    liste = []
    for i, data in enumerate(array):
        liste.append(data)
    return liste

In [17]:
# d = []
# for p, data in enumerate(ecg_labels_edited):
#     d.append(
#         {
#             'ecg_data': ecg_names[p],
#             'label': ecg_labels_enc[p]
#         }
#     )
# features_names = pd.DataFrame(d)


In [18]:
# Numerical & one-hot encode
y_train = labelencode(y_train)
y_test = labelencode(y_test)

# Convert to list
y_train = tolist(y_train)
y_test = tolist(y_test)

# Convert to array... x=(samples, time), y=(samples, encodings)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

# Classification (CNN)
- Sources:
    - https://stackoverflow.com/questions/55233377/keras-sequential-model-with-multiple-inputs
    - https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
    - https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5
    - https://www.tutorialspoint.com/keras/keras_convolution_neural_network.htm

### CNN-defaults

In [180]:
def conv_1d(filters, kernel_size, act='relu'):
    layer = tf.keras.layers.Conv1D(
        filters,
        kernel_size,
        strides=1,
        activation=act,
        use_bias=True,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        data_format="channels_last",
        padding="valid",
        # dilation_rate=1,
        # groups=1,
        # kernel_regularizer=None,
        # bias_regularizer=None,
        # activity_regularizer=None,
        # kernel_constraint=None,
        # bias_constraint=None
        )
    return layer

def maxpool(stride_num, pool_num=2):
    layer = tf.keras.layers.MaxPooling1D(
        strides=stride_num,
        pool_size=pool_num,
        padding="valid",
        # data_format="channels_last"
        )
    return layer

def fully(uni, act='relu'):
    layer = tf.keras.layers.Dense(
        units=uni,
        activation=act,
        use_bias=True,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=keras.regularizers.L2(1e-3),
        # bias_regularizer=None,
        # activity_regularizer=None,
        # kernel_constraint=None,
        # bias_constraint=None
        )
    return layer

### CNN (fan_multiscaled_2018)

In [186]:
batch_s = 128
epochen = 3
num_classes = 4
X_tr = X_train
y_tr = y_train
X_te = X_test
y_te = y_test

In [195]:
stream1 = keras.layers.Input(
    shape=(duration,1),
    #batch_size=batch_s
)
stream2 = keras.layers.Input(
    shape=(duration,1),
    #batch_size=batch_s
)

# First branch
line1 = conv_1d(64,3,act=None)(stream1)
line1 = conv_1d(64,3)(line1)
line1 = maxpool(3)(line1)
line1 = conv_1d(128,3)(line1)
line1 = conv_1d(128,3)(line1)
line1 = maxpool(3)(line1)
line1 = conv_1d(256,3)(line1)
line1 = conv_1d(256,3)(line1)
line1 = conv_1d(256,3)(line1)
line1 = maxpool(2)(line1)
line1 = conv_1d(512,3)(line1)
line1 = conv_1d(512,3)(line1)
line1 = conv_1d(512,3)(line1)
line1 = maxpool(2)(line1)
line1 = conv_1d(512,3)(line1)
line1 = conv_1d(512,3)(line1)
line1 = conv_1d(512,3)(line1)
line1_out = maxpool(2)(line1)
line1_mod = keras.Model(inputs=stream1, outputs=line1_out)

# Second branch
line2 = conv_1d(64,7,act=None)(stream2)
line2 = conv_1d(64,7)(line2)
line2 = maxpool(3)(line2)
line2 = conv_1d(128,7)(line2)
line2 = conv_1d(128,7)(line2)
line2 = maxpool(3)(line2)
line2 = conv_1d(256,3)(line2)
line2 = conv_1d(256,3)(line2)
line2 = conv_1d(256,3)(line2)
line2 = maxpool(2)(line2)
line2 = conv_1d(512,3)(line2)
line2 = conv_1d(512,3)(line2)
line2 = conv_1d(512,3)(line2)
line2 = maxpool(2)(line2)
line2 = conv_1d(512,3)(line2)
line2 = conv_1d(512,3)(line2)
line2 = conv_1d(512,3)(line2)
line2_out = maxpool(2)(line2)
line2_mod = keras.Model(inputs=stream2, outputs=line2_out)

# Combination
combined = keras.layers.Concatenate()([line1_mod.output, line2_mod.output])
combined = keras.layers.Flatten()(combined)

# Fully connected layer
line3 = fully(1024, act=None)(combined)
line3 = fully(1024)(line3)
line3 = fully(256)(line3)
outputs = fully(num_classes, act="softmax")(line3)
model = keras.Model(inputs=[stream1, stream2], outputs=outputs)

In [196]:
model.summary()

Model: "model_85"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_68 (InputLayer)          [(None, 6000, 1)]    0           []                               
                                                                                                  
 input_69 (InputLayer)          [(None, 6000, 1)]    0           []                               
                                                                                                  
 conv1d_750 (Conv1D)            (None, 5998, 64)     256         ['input_68[0][0]']               
                                                                                                  
 conv1d_763 (Conv1D)            (None, 5994, 64)     512         ['input_69[0][0]']               
                                                                                           

### Compile & train

In [197]:
# Compile
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=300,
    decay_rate=0.1)
opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    # loss_weights=None,
    # weighted_metrics=None,
    # run_eagerly=None,
    # steps_per_execution=None,
    # jit_compile=None
)

In [198]:
# Train
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='cnn_julian_{epoch}',
        save_freq='epoch',
        monitor='val_loss',
        mode='auto',
        verbose=0,
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        monitor="loss",
        patience=2,
        mode="auto",
        verbose=1,
        # min_delta=0,
        # baseline=None,
        # restore_best_weights=False,
    )
]
model.fit(
    x = X_tr,
    y = y_tr,
    #batch_size = batch_s, 
    epochs = epochen, 
    verbose = 1,
    validation_data = (X_te, y_te),
    callbacks = callbacks,
    # shuffle=True,
    # class_weight=None,
    # sample_weight=None,
    # initial_epoch=0,
    # steps_per_epoch=None,
    # validation_steps=None,
    # validation_batch_size=None,
    # validation_split=0.0,
    # validation_freq=1,
    # max_queue_size=10,
    # workers=1,
    # use_multiprocessing=False,
)

Epoch 1/3


ValueError: in user code:

    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/tljh/user/envs/tensorflow/lib/python3.8/site-packages/keras/engine/input_spec.py",

### Test & evaluate

In [ ]:
score = model.evaluate(
    x=X_te,
    y=y_te,
    verbose = 'auto',
    # batch_size=None,
    # sample_weight=None,
    # steps=None,
    # callbacks=None,
    # max_queue_size=10,
    # workers=1,
    # use_multiprocessing=False,
    # return_dict=False
) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

### Model output

In [ ]:
#Save model
Model.save(
    filepath="cnn_julian.tf",
    include_optimizer=True,
    save_format="tf",
    # overwrite=True,
    # signatures=None,
    # options=None,
    # save_traces=True,
)
#loaded_model = tf.keras.models.load_model('/tmp/model')

In [ ]:
#Model statistics
model.summary()
print(history.history)
#model.get_weights()

In [2]:
#Plot model
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    rankdir="TB",
    dpi=80,
    show_shapes=True,
    layer_range=None,
    show_layer_names=False,
    
    # Misc.
    # show_dtype=True,
    # show_layer_activations=True,
    # expand_nested=False,
)

NameError: name 'tf' is not defined

### Predict & run

In [ ]:
pred = model.predict(
    x = X_te,
    batch_size=None,
    verbose="auto"
    
    # Misc.
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)
pred = np.argmax(pred, axis = 1)[:5] 
label = np.argmax(y_test,axis = 1)[:5] 

print(pred)
print(label)

## Neuronales Netz aufstellen
- [ ] Testlauf mit niedriger Epoche: verbose=0 in fit, falls Fehlermeldung

## Tuningprozess
- https://keras.io/keras_tuner/
- https://keras.io/api/keras_tuner/tuners/
- https://keras.io/guides/keras_tuner/getting_started/
- https://keras.io/api/keras_tuner/hyperparameters/
- https://keras.io/getting_started/intro_to_keras_for_engineers/#finding-the-best-model-configuration-with-hyperparameter-tuning
- 10 fold Crossvalidation
- [08 - Data Science & Machine Learning](:/67056066e2104708bcc4c6d27239e051)
- Auch ROC, Confusion Matrix und Learning Curves ausleiten... (TP, TN, ...):
    - https://keras.io/api/metrics/classification_metrics/
    - https://keras.io/api/metrics/
    - https://keras.io/api/metrics/accuracy_metrics/
    - https://keras.io/api/metrics/probabilistic_metrics/
    - https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178
- [ ] Auf Supercomputer auslegen (Anleitung s. Moodle)
    - https://keras.io/getting_started/faq/#how-can-i-train-a-keras-model-on-multiple-gpus-on-a-single-machine
    - https://keras.io/getting_started/faq/#how-can-i-train-a-keras-model-on-tpu
    - https://keras.io/api/callbacks/backup_and_restore/
    - https://keras.io/getting_started/intro_to_keras_for_engineers/#speeding-up-training-with-multiple-gpus
    - https://keras.io/getting_started/intro_to_keras_for_engineers/#doing-preprocessing-synchronously-ondevice-vs-asynchronously-on-host-cpu
- [ ] weitere Trainingsdaten durchgehen (s. andere)

## Sonstige
- [ ] Normalisieren:
    - https://keras.io/api/layers/preprocessing_layers/numerical/normalization/
    - https://keras.io/api/layers/normalization_layers/batch_normalization/
- [ ] Rebalance: https://keras.io/examples/structured_data/imbalanced_classification/
- [ ] Schnitttechnik der Daten (nur R-Peaks cutten)
- [ ] Alternative Architekturen (notfalls einfach auch nur Code durchgehen, um zu ergänzen):
    - https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    - https://keras.io/examples/timeseries/timeseries_classification_transformer/
    - https://keras.io/api/data_loading/timeseries/